In [117]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

In [118]:
df = pd.read_csv('universities-clean.csv')

In [119]:
df.head()

,Country,University,Founded,Type,Enrollment,link
0,Albania,University of Tirana,1957,Public,35000,https://en.wikipedia.org/wiki/Albania
1,Algeria,Constantine University,1978,Public,85000,https://en.wikipedia.org/wiki/Algeria
2,Angola,Agostinho Neto University,1962,Public,29827,https://en.wikipedia.org/wiki/Angola
3,Argentina,University of Buenos Aires,1821,Public,311175,https://en.wikipedia.org/wiki/Argentina
4,Australia,Monash University,1958,Public,73807,https://en.wikipedia.org/wiki/Australia


In [120]:
conn = pymysql.Connect(
    host = 'localhost',
    port = 3307,
    password = 'amdariuserpassword',
    user = 'amdariuser',
    database = 'amdaridb'
)

In [121]:
cursor = conn.cursor()

In [122]:
cursor.execute('SHOW TABLES;')

1

In [123]:
cursor.fetchall()

(('university',),)

In [124]:
create_query_file = open('../sql/mysql_create_table.sql')
create_query = create_query_file.read()
create_query


'CREATE TABLE university (\n    id BIGINT AUTO_INCREMENT NOT NULL,\n    country VARCHAR(255) NOT NULL,\n    name VARCHAR(255) NOT NULL,\n    founded INT NOT NULL,\n    type VARCHAR(255) NOT NULL,\n    enrollment BIGINT NOT NULL,\n    link VARCHAR(255) NOT NULL,\n\n    UNIQUE KEY unique_combination (country, name),\n    PRIMARY KEY (id)\n);\n\n\n'

In [ ]:
cursor.execute(create_query)

In [ ]:
data = list(df.itertuples(index=None, name=None))
print(data)

[('Albania', 'University of Tirana', 1957, 'Public', 35000, 'https://en.wikipedia.org/wiki/Albania'), ('Algeria', 'Constantine University', 1978, 'Public', 85000, 'https://en.wikipedia.org/wiki/Algeria'), ('Angola', 'Agostinho Neto University', 1962, 'Public', 29827, 'https://en.wikipedia.org/wiki/Angola'), ('Argentina', 'University of Buenos Aires', 1821, 'Public', 311175, 'https://en.wikipedia.org/wiki/Argentina'), ('Australia', 'Monash University', 1958, 'Public', 73807, 'https://en.wikipedia.org/wiki/Australia'), ('Austria', 'University of Vienna', 1365, 'Public', 91000, 'https://en.wikipedia.org/wiki/Austria'), ('Bangladesh', 'National University, Bangladesh', 1992, 'Public', 2097182, 'https://en.wikipedia.org/wiki/Bangladesh'), ('Belarus', 'Belarusian State University', 1921, 'Public', 25500, 'https://en.wikipedia.org/wiki/Belarus'), ('Belgium', 'KU Leuven', 1425, 'Private, state-funded', 55484, 'https://en.wikipedia.org/wiki/Belgium'), ('Bosnia and Herzegovina', 'University of S

In [ ]:
engine = create_engine('mysql+pymysql://',creator=lambda:conn)

In [ ]:
pd.read_sql("SELECT* FROM university;",con=engine)

In [ ]:
merge_query_file = open('../sql/mysql_upsert.sql')
merge_query= merge_query_file.read()
print(merge_query)
print(data)

INSERT INTO university (country, `name`, founded, `type`, enrollment, link)
VALUES (%s, %s, %s, %s, %s, %s)
ON DUPLICATE KEY UPDATE
    founded = VALUES(founded),
    `type` = VALUES(`type`),
    enrollment = VALUES(enrollment),
    link = VALUES(link);


[('Albania', 'University of Tirana', 1957, 'Public', 35000, 'https://en.wikipedia.org/wiki/Albania'), ('Algeria', 'Constantine University', 1978, 'Public', 85000, 'https://en.wikipedia.org/wiki/Algeria'), ('Angola', 'Agostinho Neto University', 1962, 'Public', 29827, 'https://en.wikipedia.org/wiki/Angola'), ('Argentina', 'University of Buenos Aires', 1821, 'Public', 311175, 'https://en.wikipedia.org/wiki/Argentina'), ('Australia', 'Monash University', 1958, 'Public', 73807, 'https://en.wikipedia.org/wiki/Australia'), ('Austria', 'University of Vienna', 1365, 'Public', 91000, 'https://en.wikipedia.org/wiki/Austria'), ('Bangladesh', 'National University, Bangladesh', 1992, 'Public', 2097182, 'https://en.wikipedia.org/wiki/Bangladesh'), 

In [ ]:
cursor.executemany(merge_query, data)


69

In [ ]:
conn.commit

<bound method Connection.commit of <pymysql.connections.Connection object at 0x128c44e50>>

In [ ]:
pd.read_sql("SELECT * FROM university;",con=engine)